In [9]:

import itertools
import os
import pandas as pd
import pickle
import pysam
import sys

from collections import Counter, defaultdict
from tqdm import tqdm

In [1]:
%%bash
zcat /wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/SF_asso_S1_R2_001.fastq.gz | wc -l 

649968020


In [2]:
PROJECT_DIR = "/wynton/group/ahituv/MPRAflow"
FASTQ_IN = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/SF_asso_S1_R2_001.fastq.gz"
N_FASTQ_F = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/count_fastq.txt"
BAMFILE = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/s_merged.bam" 
N_BAMFILE = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/count_merged.txt"
PREFIX = "null_mpra" 
MPAQ = 30 
BASEQ = 30 
cigar= "n" 

In [ ]:
# CMD python ori_map_barcodes.py data_assoc/

# verify trailing arguments
PROJECT_DIR = sys.argv[1]
FASTQ_IN = sys.argv[2]
N_FASTQ_F = sys.argv[3]
BAMFILE = sys.argv[4]
N_BAMFILE = sys.argv[5]
prefix = sys.argv[6]
 = sys.argv[7]
baseq = sys.argv[8]
cigar = sys.argv[9]

In [3]:
print(PROJECT_DIR)
print(FASTQ_IN)
#print(n_fastq)
print(BAMFILE)
#print(n_bam)

if cigar=='n':
    cigar=''


print(N_FASTQ_F)
print(N_BAMFILE)


/wynton/group/ahituv/MPRAflow
/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/SF_asso_S1_R2_001.fastq.gz
/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/s_merged.bam
/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/count_fastq.txt
/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/count_merged.txt


In [4]:

def get_coords_to_barcodes(fastq_in, n_fastq,bamfile,n_bam,mapq=30,baseq=30,cigar=''):
    """#map coords to BCs 
        filter based on given read/map quality 
        and an exact cigar string if provided
    """
    coords_to_barcodes_fn = f'{PREFIX}_coords_to_barcodes.pickle'

    if not os.path.exists(coords_to_barcodes_fn):

        # load fastq
        fastq = pysam.FastxFile(fastq_in)
        print("n fastq", n_fastq)
        n_fastq_records = float(n_fastq)/4 # (wc -l) / 4
        
        #bam = pysam.AlignmentFile(f'{PROJECT_DIR}/library.bam', 'rb')

        # load bam
        bam = pysam.AlignmentFile(bamfile, 'rb')
        n_bam_records = float(n_bam)
        #print((bam))   
 
        query_to_coords = {}
        bad_pairs, poor_quality = 0, 0
        print('start')
        
        #get names of the aligned reads that havae a high enough quality and match sequence
        for i, read in tqdm(enumerate(bam), 'paired-end reads', total = n_bam_records):
            #print(read, read.cigarstring, read.reference_name, isinstance(read.reference_name, str))
            
            #if not read.is_read1:
            #    continue
            
            #if not read.is_proper_pair:
            #    bad_pairs += 1
            #    continue
            
            if read.mapping_quality < int(MPAQ):
                poor_quality += 1
                continue
            #only save exact matches and high read quality 		
            #if read.cigarstring == '146M':
            #if read.cigarstring == '179M' or read.cigarstring == '178M':
            
            ## filter reads with too low of map quality and exact cigar match if provided
            if read.mapping_quality > int(MPAQ):
                if cigar == "":
                    if isinstance(read.reference_name, str):
                        query_to_coords[read.query_name] = read.reference_name
                else:
                    if read.cigarstring==cigar:
                        if isinstance(read.reference_name, str):
                            query_to_coords[read.query_name] = read.reference_name           

        print(f'bad pairs: {bad_pairs} poor quality: {poor_quality}')
  
        print("\nstart")
        coords_to_barcodes = defaultdict(list)
        ###get get barcodes for the aligned reads that passed QC
        for i, barcode in tqdm(enumerate(fastq), 'barcodes', total = n_fastq_records):
            #print(i)
            #print(barcode.quality)
            #print(barcode.name)
            min_base_quality = min(map(ord, barcode.quality))
            
            #filter BCs with poor base quality
            if barcode.name in query_to_coords and min_base_quality >= int(baseq):
                coord = query_to_coords[barcode.name]
                coords_to_barcodes[coord].append(barcode.sequence)
                #print(coord)
        #get unique set of BCs
        #for k, v in coords_to_barcodes.items():   
            #print('set')
            #print(len(set(v)))
            #print(len(v))
        pickle.dump(coords_to_barcodes, open(coords_to_barcodes_fn, 'wb'))
    return pickle.load(open(coords_to_barcodes_fn, 'rb'))

def save_barcodes_per_candidate(d, fn):
    (
        pd.Series(d, name = 'n_barcodes')
        .str.len()
        .rename_axis('coord')
        .reset_index()
        .to_feather(fn)
    )


def save_non_repeated_barcodes_per_candidate(d, fn):
    save_barcodes_per_candidate(
        pd.Series(d, name = 'n_barcodes').apply(set),
        fn
    )

In [5]:
def readFile(infile):
    with open(infile, 'r') as file:
        read_file = file.read().replace('\n','')
    return read_file

In [6]:
#PROJECT_DIR = '/ye/yelabstore3/gracieg/scMPRA/pilot_mixing/slimsdata.genomecenter.ucdavis.edu/Data/96qdysy9rg/Unaligned/Project_NAFI_FI48'
def main(argv):

IndentationError: expected an indented block (2213363817.py, line 2)

In [12]:
    n_fastq = readFile(N_FASTQ_F)
    n_bam = readFile(N_BAMFILE)
    #f=open(n_fastq_f, 'r')
    #n_fastq=f.read().replace('\n','')
    print('counts')
    print(n_bam)
    print(n_fastq)

counts
147730701
649968020


In [13]:
    coords_to_barcodes = get_coords_to_barcodes(FASTQ_IN, n_fastq,BAMFILE,n_bam, PREFIX)

In [11]:
    save_barcodes_per_candidate(coords_to_barcodes, f'{prefix}_barcodes_per_candidate.feather')
    save_non_repeated_barcodes_per_candidate(coords_to_barcodes, f'{prefix}_barcodes_per_candidate-no_repeats.feather')

NameError: name 'prefix' is not defined

In [ ]:
counter = Counter(itertools.chain.from_iterable(coords_to_barcodes.values()))
pickle.dump(counter, open(f'{prefix}_barcode_counts.pickle', 'wb'))

jackpotted_barcodes = {barcode for barcode, count in counter.most_common(50)}
filtered_coords_to_barcodes = {}
for coord, barcodes in tqdm(coords_to_barcodes.items(), 'filtered barcodes'):
    filtered_coords_to_barcodes[coord] = list(filter(lambda x: x not in jackpotted_barcodes, barcodes))

save_barcodes_per_candidate(filtered_coords_to_barcodes, f'{prefix}_barcodes_per_candidate-no_jackpots.feather')
save_non_repeated_barcodes_per_candidate(filtered_coords_to_barcodes, f'{prefix}_barcodes_per_candidate-no_repeats-no_jackpots.feather')